In [1]:
import scispacy
import pandas as pd
import re

In [153]:
def explode_protocol_summary(text):
    items = re.findall("<b>(.*?)</b>", text)
    values = re.findall("</b>(.*?)<b>", text)
    protocol = "standard"
    body_part = oral_contrast = iv_contrast = ""
    try:
        if len(values) != len(items):
            values.append(re.findall("(?s:.*)</b>(.*?)$", text)[-1].split("|")[0])
    except:
        pass
    processed_values = []
    for s in values:
        processed_values.append(",".join(string for string in s.split("|") if len(string) > 0))
    if len(items) == 3:
        items.insert(0, "Protocol:")
        processed_values.insert(0, "Standard")
    for i in range(len(items)):
        if items[i] == "Protocol:":
            protocol = processed_values[i]
        if items[i] == "Body Part:":
            body_part = processed_values[i]
        if items[i] == "Oral Contrast:":
            oral_contrast = processed_values[i]
        if items[i] == "IV Contrast:":
            iv_contrast = processed_values[i]
    return (protocol, body_part, oral_contrast, iv_contrast)

In [166]:
data = pd.read_csv("Past5YCT2.csv", delimiter=",")
filter_rows = ["reasonofstudy", "StudyDesc", "procedurecode", "protocolsummary"]
data = data[filter_rows]

# preprocessing
data = data.dropna()

# drop rows that doesn't have Oral Contrast / IV Contrast
data = data[data["protocolsummary"].str.contains("IV Contrast")]


data["Protocol"], data["Body Part"], data["Oral Contrast"], data["IV Contrast"] = zip(*data.apply(lambda x: explode_protocol_summary(x["protocolsummary"]), axis=1))
data = data[data["IV Contrast"] != ""]
data = data[(data["IV Contrast"] != "") & (data["Oral Contrast"] != "")]

In [169]:
data.head()

,reasonofstudy,StudyDesc,procedurecode,protocolsummary,Protocol,Body Part,Oral Contrast,IV Contrast
1,Reason: history of left NSGCT - eval RP recurrence/metastasis History: history of left testicular cancer,CT UPPER ABD AND PELVIS W,RADCT001,Protocol by echang2|<b>Body Part:</b>Abdomen/Pelvis|<b>Oral Contrast:</b>Omnipaque 300/Water|<b>IV Contrast:</b>Omnipaque 350 - 120 cc,Standard,Abdomen/Pelvis,Omnipaque 300/Water,Omnipaque 350 - 120 cc
4,"Reason: History of locally advanced prostate cancer, assess for metastatic cancer History: locally urinary symptoms",CT UPPER ABD AND PELVIS W,RADCT001,Protocol by echang2|<b>Body Part:</b>Abdomen/Pelvis|<b>Oral Contrast:</b>Omnipaque 300/Water|<b>IV Contrast:</b>Omnipaque 350 - 120 cc,Standard,Abdomen/Pelvis,Omnipaque 300/Water,Omnipaque 350 - 120 cc
7,Reason: eval for g-tube leak History: follow up scan,CT CHEST ABDOMEN PELVIS WO,RADCT023,Protocol by nflorence|<b>Body Part:</b>Chest/Abdomen/Pelvis|<b>Oral Contrast:</b>Omnipaque 300/Water|<b>IV Contrast:</b>None,Standard,Chest/Abdomen/Pelvis,Omnipaque 300/Water,None
18,"Reason: History of known intraabdominal lymphadenopathy, chronic hepatitis B with advanced fibrosis, assess for stability, exclude hepatic lesions History: History of known intraabdominal lymphadenopathy, chronic hepatitis B with advanced fibrosis, assess for stability, exclude hepatic lesions",CT UPPER ABD AND PELVIS WWO,RADCT026,Protocol by echang2|<b>Protocol:</b>|Dedicated Liver Protocol|CT Dedicated Liver|<b>Body Part:</b>Abdomen/Pelvis|<b>Oral Contrast:</b>None|<b>IV Contrast:</b>Omnipaque 350 - 120 cc|<b>Comments:</b>pelvis in portal venous,"Dedicated Liver Protocol,CT Dedicated Liver",Abdomen/Pelvis,None,Omnipaque 350 - 120 cc
20,"Reason: h/o met head and neck ca, compare to previous, measurements pls History: none",CT CHEST ABDOMEN PELVIS W,RADCT022,Protocol by nflorence|<b>Body Part:</b>Chest/Abdomen/Pelvis|<b>Oral Contrast:</b>Omnipaque 300/Water|<b>IV Contrast:</b>Omnipaque 350 - 120 cc,Standard,Chest/Abdomen/Pelvis,Omnipaque 300/Water,Omnipaque 350 - 120 cc
